In [1]:
import numpy as np
import pandas as pd
import math
import operator

In [2]:
data = pd.read_csv('/home/becomp/Downloads/data.csv')

In [3]:
data

,Age,Income,Gender,Marital Status,Buys
0,<21,High,Male,Single,No
1,<21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,>35,Medium,Male,Single,Yes
4,>35,Low,Female,Single,Yes
5,>35,Low,Female,Married,No
6,21-35,Low,Female,Married,Yes
7,<21,Medium,Male,Single,No
8,<21,Low,Female,Married,Yes
9,>35,Medium,Female,Single,Yes


In [4]:
for i,row in data.iterrows():
    if row['Age'] == '<21':
        data.at[i,'Age']=0
    elif row['Age'] == '21-35':
        data.at[i,'Age'] = 1
    else:
        data.at[i,'Age'] = 2

In [5]:
for i,row in data.iterrows():
    if row['Income'] == 'Low':
        data.at[i,'Income'] = 0
    elif row['Income'] == 'Medium':
        data.at[i,'Income'] = 1
    else:
        data.at[i,'Income'] = 2

In [6]:
for i,row in data.iterrows():
    if row['Gender'] == 'Male':
        data.at[i,'Gender'] = 0
    else:
        data.at[i,'Gender'] = 1
        
    if row['Marital Status'] == 'Single':
        data.at[i,'Marital Status'] = 0
    else:
        data.at[i,'Marital Status'] = 1
        
    if row['Buys'] == 'No':
        data.at[i,'Buys'] = 0
    else:
        data.at[i,'Buys'] = 1

In [7]:
data

,Age,Income,Gender,Marital Status,Buys
0,0,2,0,0,0
1,0,2,0,1,0
2,1,2,0,0,1
3,2,1,0,0,1
4,2,0,1,0,1
5,2,0,1,1,0
6,1,0,1,1,1
7,0,1,0,0,0
8,0,0,1,1,1
9,2,1,1,0,1


In [8]:
#class for node
class Node(object):
    def __init__(self,data):
        self.left = None
        self.ldata = None
        self.mid = None
        self.mdata = None
        self.right = None
        self.rdata = None
        self.data = data
        self.entropy = None
        self.split_parameter = None
        self.isLeaf = False
        self.numChild = 0
    def calc_entropy(self,a,b):
    #a,b are integers
        t = a + b
        
        if a==0 or b==0:
            return 0
        return -(a/t)*math.log2(a/t) - (b/t)*math.log2(b/t)
    def split(self,columns):
        
        n = len(data)
        
        #1. calculate base entropy
        n_yes = data[data['Buys']==1]['Buys'].count()
        n_no = data[data['Buys']==0]['Buys'].count()
        base_entropy = self.calc_entropy(n_yes,n_no)
        
        #if base entropy is 0, generate leaf node
        if base_entropy==0:
            return 'Leaf'
        c_entropys = {}
        
        if len(columns)==0:
            return 'Leaf'
        print('columns:',columns)
        #2. Calculate conditional entropy for each column
        for col in columns:
            unique_values = list(data[col].unique())
            c_e = 0
            for val in unique_values:
                num_yes = data[(data[col]==val) & (data['Buys']==1)]['Buys'].count()
                num_no = data[(data[col]==val) & (data['Buys']==0)]['Buys'].count()
                c_e = c_e + ((num_no+num_yes)/n)*self.calc_entropy(num_yes,num_no)
                
            c_entropys[col] = c_e
        
        print(base_entropy)
        print(c_entropys)
        
        #3. choose the column with max info gain
        igs = {}
        for item in c_entropys:
            igs[item] = base_entropy - c_entropys[item]
        
        print(igs)
        max_key = max(igs.items(), key=operator.itemgetter(1))[0]
        
        self.split_parameter = max_key
        
        
        return max_key
            
            
        
        

In [45]:
#create root node
root = Node(data)

In [46]:
columns = list(data)

In [47]:
columns

['Age', 'Income', 'Gender', 'Marital Status', 'Buys']

In [48]:
columns.remove('Buys')

In [49]:
columns

['Age', 'Income', 'Gender', 'Marital Status']

In [50]:
def IDR3(root,cols):
    col = root.split(cols)
    print(cols)
    #if len(cols) == 0:
    #    return
    if col=='Leaf':
        #make the current node a leaf node
        root.isLeaf = True
        root.data = list(root.data['Buys'].unique())[0]
        print('unique:',root.data)
        return
    
    unique_vals = list(root.data[col].unique())
    cols.remove(col)
    
    if len(unique_vals)==3:
        root.ldata = unique_vals[0]
        root.mdata = unique_vals[1]
        root.rdata = unique_vals[2]

        #create nodes
        l_node = Node(root.data[root.data[col]== root.ldata])
        m_node = Node(root.data[root.data[col]== root.mdata])
        r_node = Node(root.data[root.data[col]== root.rdata])

        #connect root to the nodes

        root.left = l_node
        root.mid = m_node
        root.right =  r_node
        IDR3(l_node,cols)
        IDR3(m_node,cols)
        IDR3(r_node,cols)
        
    elif len(unique_vals)==2:
        root.ldata = unique_vals[0]
        root.rdata = unique_vals[1]

        l_node = Node(root.data[root.data[col]== root.ldata])
        r_node = Node(root.data[root.data[col]== root.rdata])
        root.left = l_node
        root.right = r_node
        IDR3(l_node,cols)
        IDR3(r_node,cols)

root.calc_entropy(2,0)

In [51]:
IDR3(root,columns)

columns: ['Age', 'Income', 'Gender', 'Marital Status']
0.9402859586706309
{'Age': 0.6935361388961918, 'Income': 0.9110633930116763, 'Gender': 0.7884504573082896, 'Marital Status': 0.9241743523004413}
{'Age': 0.2467498197744391, 'Income': 0.029222565658954647, 'Gender': 0.15183550136234136, 'Marital Status': 0.0161116063701896}
['Age', 'Income', 'Gender', 'Marital Status']
columns: ['Income', 'Gender', 'Marital Status']
0.9402859586706309
{'Income': 0.9110633930116763, 'Gender': 0.7884504573082896, 'Marital Status': 0.9241743523004413}
{'Income': 0.029222565658954647, 'Gender': 0.15183550136234136, 'Marital Status': 0.0161116063701896}
['Income', 'Gender', 'Marital Status']
columns: ['Income', 'Marital Status']
0.9402859586706309
{'Income': 0.9110633930116763, 'Marital Status': 0.9241743523004413}
{'Income': 0.029222565658954647, 'Marital Status': 0.0161116063701896}
['Income', 'Marital Status']
columns: ['Marital Status']
0.9402859586706309
{'Marital Status': 0.9241743523004413}
{'Mari

In [52]:
def BFS(root):
    q = []
    q.append(root)
    
    
    while(len(q)!=0):
        n = q[0]
        q.pop(0)
        print(n.split_parameter)
        if n.left is not None and n.left.split_parameter is not None:
            q.append(n.left)
            
        if n.mid is not None and n.mid.split_parameter is not None:
            q.append(n.mid)
        if n.right is not None and n.right.split_parameter is not None:
            q.append(n.right)

In [53]:
BFS(root)

Age
Gender
Income
Marital Status


In [54]:
def traverse_tree(root,data):
    #data will be in the form of a dictionary and preprocessed
    node = root
    while(node.isLeaf==False):
        print(node.split_parameter)
        x = data[node.split_parameter]
        if node.ldata==x:
            node=node.left
        elif node.mdata==x:
            node = node.mid
        else:
            node = node.right
    print(node.data)

In [ ]:
data.head(4)

In [ ]:
sample = {'Age':0,'Income':2,'Gender':0,'Marital Status':0}

In [ ]:
sample[root.split_parameter]

In [ ]:
traverse_tree(root,{'Age':2,'Income':2,'Gender':0,'Marital Status':0})

In [ ]:
s_dict = {'a':45}

In [ ]:
s_dict.items()

In [ ]:
max(s_dict.items(), key=operator.itemgetter(1))[0]

In [ ]:
root.left.mid is None

In [25]:
s = data[data['Buys']==1]

In [29]:
list(s['Buys'].unique())[0]

1